In [1]:
!pwd

/Users/sarthakarora/Insight/insight-borderanalysis/src


In [2]:
import csv
csvpath = '../input/Border_Crossing_Entry_Data.csv'
csvpath2 = '../input/Border_Crossing_Entry_Data2.csv'
data = {}
data2 = {}
with open(csvpath) as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for i,row in enumerate(readCSV):
        if i == 0:
            cols = row
            for colname in row:
                data[colname] = []
        else:
            for j,value in enumerate(row):
                data[cols[j]].append(value)
                
with open(csvpath2) as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for i,row in enumerate(readCSV):
        if i == 0:
            cols = row
            for colname in row:
                data2[colname] = []
                print(colname)
        else:
            for j,value in enumerate(row):
                data2[cols[j]].append(value)
            
        

Port Name
State
Port Code
Border
Date
Measure
Value


In [3]:
keys = ['Port Name','State','Port Code','Border','Date','Measure','Value']

In [4]:
# import json
# print (json.dumps(data,indent=4))
# # print(data)

In [5]:
# ## zipped instances are meant for only one iteration. They get exhausted after that
# mapped = zip(data['State'], data['Port Code'],data['Border'], data['Date'],data['Measure'], data['Value'],data['Location'])

In [6]:
# set(data2['Border']) # to print unique elements in list of borders

{'US-Canada Border', 'US-Mexico Border'}

In [7]:
set(data2['State'])

{'AK',
 'AZ',
 'CA',
 'ID',
 'ME',
 'MI',
 'MN',
 'MT',
 'ND',
 'NM',
 'NY',
 'OH',
 'TX',
 'VT',
 'WA'}

In [8]:
sorted(set(data2['Measure']),reverse=True)

['Trucks',
 'Truck Containers Full',
 'Truck Containers Empty',
 'Trains',
 'Train Passengers',
 'Rail Containers Full',
 'Rail Containers Empty',
 'Personal Vehicles',
 'Personal Vehicle Passengers',
 'Pedestrians',
 'Buses',
 'Bus Passengers']

In [9]:
# type(list(set(data2['Date']))[0])
set(data['Date'])

{'01/01/2019 12:00:00 AM', '02/01/2019 12:00:00 AM', '03/01/2019 12:00:00 AM'}

In [10]:
def groupby_column(dictionary, column):
    temp = {}
    for index, key in enumerate(dictionary[column]):
        try:
            temp[key].append(index)
        except:
            temp[key] = [index]
    return temp

In [11]:
# filtering on dates. getting indices
groupby_date = groupby_column(data2,keys[4])
groupby_measure = groupby_column(data2,keys[5])
groupby_border = groupby_column(data2,keys[3])

# groupby_date = groupby_column(data,keys[4])
# groupby_measure = groupby_column(data,keys[5])
# groupby_border = groupby_column(data,keys[3])

In [12]:
# groupby_border

In [13]:
# sorted(groupby_date,reverse=True)

In [15]:
reportkeys = ['Border','Date','Measure','Value','Average']
f2 = {}
for key in reportkeys:
    f2[key] = []

for date in sorted(groupby_date,reverse=True):
    for measure in sorted(groupby_measure):
        for border in sorted(groupby_border):
            intersection = list(set(groupby_measure[measure]) & set(groupby_date[date]) & set(groupby_border[border]))
            if len(intersection) != 0:
#                 res = [int(data[keys[6]][i]) for i in intersection]
                res = [int(data2[keys[6]][i]) for i in intersection]
#                 print(border,date,measure,sum(res))
                f2[reportkeys[0]].append(border)
                f2[reportkeys[1]].append(date)
                f2[reportkeys[2]].append(measure)
                f2[reportkeys[3]].append(sum(res))
                f2[reportkeys[4]].append(0)
#             print(res, sum(res))

In [ ]:
# f2

In [16]:
def proper_round(num):
#     print(num)
    temp = str(num)
    temp = temp.split('.')    
    if int(temp[1]) >= 5:
        return int(temp[0])+1
    else:
        return int(temp[0])
    

In [17]:
def compute_movavg(f2):
    for measure in set(f2[reportkeys[2]]):
        firstoccurrence = 0
        idx = []
        for m in range(len(f2[reportkeys[2]])-1,-1,-1):
            if f2[reportkeys[2]][m] == measure and firstoccurrence == 0:
                firstoccurrence = 1
                f2[reportkeys[4]][m] = 0
                idx.append(m)
            elif f2[reportkeys[2]][m] == measure and firstoccurrence == 1:
                tobesummed = [f2[reportkeys[3]][i] for i in idx]
                movavg = sum(tobesummed)/len(tobesummed)
                idx.append(m)
                movavg = proper_round(movavg)
                f2[reportkeys[4]][m] = movavg
    return f2
            
#         print(f2[reportkeys[1]][m])
#         print(m)

In [18]:
f2 = compute_movavg(f2)

In [19]:
groupby_value = groupby_column(f2, reportkeys[3])
groupby_average = groupby_column(f2, reportkeys[4])
groupby_date = groupby_column(f2, reportkeys[1])
groupby_measure = groupby_column(f2, reportkeys[2])
groupby_border = groupby_column(f2, reportkeys[0])

In [ ]:
# groupby_border

In [20]:
f3 = {}
for key in reportkeys:
    f3[key] = []

filename = '../output/report2.csv'
# # writing to csv file 
# with open(filename, 'w') as csvfile: 
#     # creating a csv writer object 
#     csvwriter = csv.writer(csvfile) 
      
#     # writing the fields 
#     csvwriter.writerow(reportkeys) 
      
    
for date in groupby_date:
    for value in sorted(groupby_value,reverse = True):
        for measure in groupby_measure:
            for border in groupby_border:
                for average in groupby_average:
                    intersection = list(set(groupby_measure[measure]) & set(groupby_date[date]) & set(groupby_border[border]) & set(groupby_value[value]) & set(groupby_average[average]))
                    if len(intersection) != 0:
                        row = [border,date,measure,value,average]
                        # writing the data rows 
#                             print(row)
#                             csvwriter.writerow(row)
                        f3[reportkeys[0]].append(border)
                        f3[reportkeys[1]].append(date)
                        f3[reportkeys[2]].append(measure)
                        f3[reportkeys[3]].append(value)
                        f3[reportkeys[4]].append(average)

KeyboardInterrupt: 

In [ ]:
filename = '../output/report2.csv'
# # writing to csv file 
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
      
    # writing the fields 
    csvwriter.writerow(reportkeys)
    mapped = zip(f3['Border'], f3['Date'],f3['Measure'],f3['Value'],f3['Average'])
    
    csvwriter.writerows(mapped) 
#     for data in f3.items():
        